Sobre la colección Vaswani, ejecute los stemmers Porter y Lancaster provistos en el módulo nltk.stem. Compare: cantidad de tokens únicos resultantes, resultado 1 a 1 y tiempo de ejecución para toda la colección. ¿Qué conclusiones puede obtener de la ejecución de uno y otro?

In [11]:

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import re
import time


def leer_docs(archivo):
    documentos = {}  

    with open(archivo, "r", encoding="utf-8") as f:
        contenido = f.read()  
    
        docs = re.findall(r"<DOC>\s*<DOCNO>(\d+)</DOCNO>\s*(.*?)\s*</DOC>", contenido, re.DOTALL)

        for doc_id, texto in docs:
            documentos[doc_id] = texto.strip()  # Guardar texto limpio

    return documentos

def sacar_acentos(texto):
    texto = re.sub(r"[áàäâã]", "a", texto)
    texto = re.sub(r"[éèëê]", "e", texto)
    texto = re.sub(r"[íìïî]", "i", texto)
    texto = re.sub(r"[óòöôõ]", "o", texto)
    texto = re.sub(r"[úùüû]", "u", texto)
    return texto

def procesarPorter(documentos):
    stemmer = PorterStemmer()
    tokens_porter = set()
    for doc_id, texto in documentos.items():
        texto = texto.lower()  # Convertir a minusculas
        texto = sacar_acentos(texto)
        texto = re.sub(r"[^a-z\s]", "", texto)  # Eliminar caracteres especiales
        tokens = texto.split()  # Tokenizar (En este caso separo por espacios)
        tokens = [stemmer.stem(token) for token in tokens]
        tokens_porter.update(tokens)
        
    return tokens_porter

def procesarLancaster(documentos):
    stemmer = LancasterStemmer()
    tokens_lancaster = set()
    for doc_id, texto in documentos.items():
        texto = texto.lower()  # Convertir a minusculas
        texto = sacar_acentos(texto)
        texto = re.sub(r"[^a-z\s]", "", texto)  # Eliminar caracteres especiales
        tokens = texto.split()  # Tokenizar (En este caso separo por espacios)
        tokens = [stemmer.stem(token) for token in tokens]
        tokens_lancaster.update(tokens)

    return tokens_lancaster

archivo_trec = "vaswani/corpus/doc-text.trec"
documentos = leer_docs(archivo_trec)

inicio = time.time()
tokens_porter = procesarPorter(documentos)
fin = time.time()
porter_time = round(fin - inicio, 2)

inicio = time.time()
tokens_lancaster = procesarLancaster(documentos)
fin = time.time()
lancaster_time = round(fin - inicio, 2)

print(f"Terminos Porter: {len(tokens_porter)}")
print(f"Terminos Lancaster: {len(tokens_lancaster)}")
print(f"Tiempo de procesamiento con Porter: {porter_time}s")
print(f"Tiempo de procesamiento con Lancaster: {lancaster_time}s")
print(f"Interseccion: {len(tokens_porter.intersection(tokens_lancaster))}")
print(f"Diferencia: {len(tokens_porter.difference(tokens_lancaster))}")

tokens_porter = sorted(list(tokens_porter))
tokens_lancaster = sorted(list(tokens_lancaster))

with open("porter_lancaster.txt", "w", encoding="utf-8") as f:
    f.write("Porter\tLancaster\n")
    for i in range(max(len(tokens_porter), len(tokens_lancaster))):
        if i < len(tokens_porter):
            f.write(f"{tokens_porter[i]}")
        if i < len(tokens_lancaster):
            f.write(f"\t{tokens_lancaster[i]}")
        f.write("\n")

Terminos Porter: 7993
Terminos Lancaster: 6877
Tiempo de procesamiento con Porter: 4.02s
Tiempo de procesamiento con Lancaster: 3.51s
Interseccion: 4760
Diferencia: 3233
